In [ ]:
# ## install finrl library
!pip install git+https://github.com/mensudza/FinRL.git

  Cloning https://github.com/mensudza/FinRL.git to /tmp/pip-req-build-_poraica
  Running command git clone -q https://github.com/mensudza/FinRL.git /tmp/pip-req-build-_poraica
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-qghdg_16/pyfolio_1f627910f2924da1ae144bfeb1419d5d
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-qghdg_16/pyfolio_1f627910f2924da1ae144bfeb1419d5d
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-qghdg_16/elegantrl_03b1fdbf4d91432b872c17e91eb05814
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-qghdg_16/elegantrl_03b1fdbf4d91432b872c17e91eb05814
     |████████████████████████████████| 234 kB 13.2 MB/s 
     |████████████████████████████████| 176 kB 73.0 MB/s 
     |████████████████████████████████| 59.6 MB 156 kB/s 
     |████████████████████████████████| 1.2 MB 67.3 MB/s 
     |████████████████████████████████

In [ ]:
import sys

sys.path.append("..")

In [ ]:
import pandas as pd
print(pd.__version__)

1.3.5


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading_cashpenalty import StockTradingEnvCashpenalty
#from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.drl_agents.stablebaselines3.models import DRLAgent as DRLAgent_sb3
from finrl.drl_agents.rllib.models import DRLAgent as DRLAgent_rllib
from finrl.drl_agents.elegantrl.models import DRLAgent as DRLAgent_erl
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


import sys
sys.path.append("../FinRL-Library")

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [ ]:
asset1 = pd.read_csv('/content/Asset01_train.csv')
asset1['tic'] = 'asset01'
asset2 = pd.read_csv('/content/Asset02_train.csv')
asset2['tic'] = 'asset02'
asset3 = pd.read_csv('/content/Asset03_train.csv')
asset3['tic'] = 'asset03'
asset4 = pd.read_csv('/content/Asset04_train.csv')
asset4['tic'] = 'asset04'
asset5 = pd.read_csv('/content/Asset05_train.csv')
asset5['tic'] = 'asset05'
f = [asset1,asset2,asset3,asset4,asset5]
train = pd.concat(f)
train

,timestamp,open,high,low,close,volume,tic
0,2010-01-04,13.854970,13.979149,13.788238,13.937369,45505,asset01
1,2010-01-05,13.936982,14.006905,13.875086,13.890270,46686,asset01
2,2010-01-06,13.891043,13.959323,13.813577,13.926440,45435,asset01
3,2010-01-07,13.926730,13.970735,13.829341,13.833886,44298,asset01
4,2010-01-08,13.833693,13.952069,13.794524,13.934854,34425,asset01
...,...,...,...,...,...,...,...
2106,2018-12-21,13.659611,13.664298,13.508989,13.603909,115700,asset05
2107,2018-12-24,13.552472,13.675214,13.518445,13.662061,97300,asset05
2108,2018-12-26,13.522105,13.590798,13.473208,13.506825,108800,asset05
2109,2018-12-27,13.665899,13.689065,13.424605,13.424605,135400,asset05


In [ ]:
asset1 = pd.read_csv('/content/Asset01_test.csv')
asset1['tic'] = 'asset01'
asset2 = pd.read_csv('/content/Asset02_test.csv')
asset2['tic'] = 'asset02'
asset3 = pd.read_csv('/content/Asset03_test.csv')
asset3['tic'] = 'asset03'
asset4 = pd.read_csv('/content/Asset04_test.csv')
asset4['tic'] = 'asset04'
asset5 = pd.read_csv('/content/Asset05_test.csv')
asset5['tic'] = 'asset05'
f = [asset1,asset2,asset3,asset4,asset5]
test = pd.concat(f)
test

,timestamp,open,high,low,close,volume,tic
0,2019-01-02,11.069159,11.118773,10.952427,10.970802,75137,asset01
1,2019-01-03,10.970706,11.035600,10.937050,11.018385,112036,asset01
2,2019-01-04,11.017805,11.043337,10.972350,11.020900,87686,asset01
3,2019-01-07,11.021093,11.104653,11.020609,11.096722,61863,asset01
4,2019-01-08,11.096529,11.106587,11.046142,11.064517,65376,asset01
...,...,...,...,...,...,...,...
229,2019-12-24,16.034499,16.131778,16.006700,16.125625,163000,asset05
230,2019-12-26,16.117883,16.259023,16.113277,16.259023,182400,asset05
231,2019-12-27,16.256304,16.414501,16.238890,16.246485,247100,asset05
232,2019-12-30,16.209359,16.443068,16.129209,16.435645,248100,asset05


In [ ]:
f = [train, test]
df = pd.concat(f)
df

,timestamp,open,high,low,close,volume,tic
0,2010-01-04,13.854970,13.979149,13.788238,13.937369,45505,asset01
1,2010-01-05,13.936982,14.006905,13.875086,13.890270,46686,asset01
2,2010-01-06,13.891043,13.959323,13.813577,13.926440,45435,asset01
3,2010-01-07,13.926730,13.970735,13.829341,13.833886,44298,asset01
4,2010-01-08,13.833693,13.952069,13.794524,13.934854,34425,asset01
...,...,...,...,...,...,...,...
229,2019-12-24,16.034499,16.131778,16.006700,16.125625,163000,asset05
230,2019-12-26,16.117883,16.259023,16.113277,16.259023,182400,asset05
231,2019-12-27,16.256304,16.414501,16.238890,16.246485,247100,asset05
232,2019-12-30,16.209359,16.443068,16.129209,16.435645,248100,asset05


In [ ]:
df['date'] = pd.to_datetime(df['timestamp'])
df['day'] = df['date'].dt.dayofweek
df

,timestamp,open,high,low,close,volume,tic,date,day
0,2010-01-04,13.854970,13.979149,13.788238,13.937369,45505,asset01,2010-01-04,0
1,2010-01-05,13.936982,14.006905,13.875086,13.890270,46686,asset01,2010-01-05,1
2,2010-01-06,13.891043,13.959323,13.813577,13.926440,45435,asset01,2010-01-06,2
3,2010-01-07,13.926730,13.970735,13.829341,13.833886,44298,asset01,2010-01-07,3
4,2010-01-08,13.833693,13.952069,13.794524,13.934854,34425,asset01,2010-01-08,4
...,...,...,...,...,...,...,...,...,...
229,2019-12-24,16.034499,16.131778,16.006700,16.125625,163000,asset05,2019-12-24,1
230,2019-12-26,16.117883,16.259023,16.113277,16.259023,182400,asset05,2019-12-26,3
231,2019-12-27,16.256304,16.414501,16.238890,16.246485,247100,asset05,2019-12-27,4
232,2019-12-30,16.209359,16.443068,16.129209,16.435645,248100,asset05,2019-12-30,0


In [ ]:
df = df.drop(columns = ['date'])
df = df.rename(columns = {'timestamp' : 'date'})
df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,13.854970,13.979149,13.788238,13.937369,45505,asset01,0
1,2010-01-05,13.936982,14.006905,13.875086,13.890270,46686,asset01,1
2,2010-01-06,13.891043,13.959323,13.813577,13.926440,45435,asset01,2
3,2010-01-07,13.926730,13.970735,13.829341,13.833886,44298,asset01,3
4,2010-01-08,13.833693,13.952069,13.794524,13.934854,34425,asset01,4
...,...,...,...,...,...,...,...,...
229,2019-12-24,16.034499,16.131778,16.006700,16.125625,163000,asset05,1
230,2019-12-26,16.117883,16.259023,16.113277,16.259023,182400,asset05,3
231,2019-12-27,16.256304,16.414501,16.238890,16.246485,247100,asset05,4
232,2019-12-30,16.209359,16.443068,16.129209,16.435645,248100,asset05,0


In [ ]:
df[df['date'] == '2017-12-29']

,date,open,high,low,close,volume,tic,day
1876,2017-12-29,11.549628,11.630190,11.543825,11.599822,84976,asset01,4
1876,2017-12-29,59.493547,60.089375,59.404172,60.000000,464478,asset02,4
1876,2017-12-29,12.382319,12.498088,12.380406,12.498088,65,asset03,4
1876,2017-12-29,46.189316,46.427645,46.115550,46.138248,18717400,asset04,4
1876,2017-12-29,17.815483,17.885653,17.802096,17.879960,141600,asset05,4


In [ ]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2010-01-04,13.854970,13.979149,13.788238,13.937369,45505,asset01,0
1,2010-01-04,79.076464,81.112217,79.076464,80.943401,263542,asset02,0
2,2010-01-04,10.693647,10.737659,10.496556,10.693647,184,asset03,0
3,2010-01-04,13.377448,13.649820,13.360425,13.564704,38409100,asset04,0
4,2010-01-04,17.785769,17.815661,17.534744,17.537128,109400,asset05,0


In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [ ]:
processed['log_volume'] = np.log(processed.volume*processed.close)
processed['change'] = (processed.close-processed.open)/processed.close
processed['daily_variance'] = (processed.high-processed.low)/processed.close
processed.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,log_volume,change,daily_variance
0,2010-01-04,13.854970,13.979149,13.788238,13.937369,45505,asset01,0,0.0,13.980427,13.847211,0.0,66.666667,100.0,13.937369,13.937369,0.0,13.360151,0.005912,0.013698
1,2010-01-04,79.076464,81.112217,79.076464,80.943401,263542,asset02,0,0.0,13.980427,13.847211,0.0,66.666667,100.0,80.943401,80.943401,0.0,16.875718,0.023065,0.025150
2,2010-01-04,10.693647,10.737659,10.496556,10.693647,184,asset03,0,0.0,13.980427,13.847211,0.0,66.666667,100.0,10.693647,10.693647,0.0,7.584586,0.000000,0.022546
3,2010-01-04,13.377448,13.649820,13.360425,13.564704,38409100,asset04,0,0.0,13.980427,13.847211,0.0,66.666667,100.0,13.564704,13.564704,0.0,20.071276,0.013805,0.021334
4,2010-01-04,17.785769,17.815661,17.534744,17.537128,109400,asset05,0,0.0,13.980427,13.847211,0.0,66.666667,100.0,17.537128,17.537128,0.0,14.467086,-0.014178,0.016018


In [ ]:
processed.tail()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,log_volume,change,daily_variance
11720,2019-12-31,10.830376,10.869738,10.829505,10.846720,32207,asset01,1,0.026092,10.833995,10.666697,58.961256,215.421134,45.579579,10.726664,10.714886,0.541491,12.763802,0.001507,0.003709
11721,2019-12-31,61.251243,61.449854,60.208543,60.635554,494535,asset02,1,1.087590,62.434038,56.725847,56.988456,95.087645,26.197404,58.633567,56.756539,0.541491,17.216255,-0.010154,0.020472
11722,2019-12-31,14.483354,14.575202,14.483354,14.537888,301,asset03,1,0.101125,14.536995,13.817005,59.790819,239.782231,48.414824,14.120743,14.162553,0.541491,8.383868,0.003751,0.006318
11723,2019-12-31,87.283865,87.851308,87.102279,87.811583,18369400,asset04,1,1.472846,89.523757,82.436618,66.262353,100.229108,28.410028,85.252603,82.060141,0.541491,21.201390,0.006010,0.008530
11724,2019-12-31,16.415977,16.498646,16.384219,16.490250,217400,asset05,1,0.152122,16.625837,15.438404,59.945427,140.582462,25.081756,15.915616,15.929588,0.541491,15.092264,0.004504,0.006939


In [ ]:
train_start = '2010-01-04'
train_end = '2019-01-02'
val_test_start = '2019-01-02'
val_test_end = '2020-01-02'
train = data_split(processed, train_start,train_end)
trade = data_split(processed, val_test_start,val_test_end)
print(len(train))
print(len(trade))

10555
1170


In [ ]:
print(StockTradingEnvCashpenalty.__doc__)


    A stock trading environment for OpenAI gym
    This environment penalizes the model for not maintaining a reserve of cash.
    This enables the model to manage cash reserves in addition to performing trading procedures.
    Reward at any step is given as follows
        r_i = (sum(cash, asset_value) - initial_cash - max(0, sum(cash, asset_value)*cash_penalty_proportion-cash))/(days_elapsed)
        This reward function takes into account a liquidity requirement, as well as long-term accrued rewards.
    Parameters:
        df (pandas.DataFrame): Dataframe containing data
        buy_cost_pct (float): cost for buying shares
        sell_cost_pct (float): cost for selling shares
        hmax (int, array): maximum cash to be traded in each trade per asset. If an array is provided, then each index correspond to each asset
        discrete_actions (bool): option to choose whether perform dicretization on actions space or not
        shares_increment (int): multiples number of shares ca

In [ ]:
information_cols = ['daily_variance', 'change', 'log_volume', 'close','day', 
                    'macd', 'rsi_30', 'cci_30', 'dx_30']

e_train_gym = StockTradingEnvCashpenalty(df = train,initial_amount = 100000,hmax = 30000, 
                                turbulence_threshold = .18, 
                                currency='$',
                                buy_cost_pct=4e-4,
                                sell_cost_pct=0,
                                #cache_indicator_data=True,
                                daily_information_cols = information_cols, 
                                print_verbosity = 50, 
                                cash_penalty_proportion=0.6,
                                random_start = True,
                                patient = True
                                )

e_trade_gym = StockTradingEnvCashpenalty(df = trade,initial_amount = 100000,hmax = 30000, 
                                turbulence_threshold = .18, 
                                currency='$',
                                buy_cost_pct=4e-4,
                                sell_cost_pct=0,
                                #cache_indicator_data=True,
                                cash_penalty_proportion=0.6,
                                daily_information_cols = information_cols, 
                                print_verbosity = 50, 
                                patient = True, 
                                random_start = False)

caching data
data cached!
caching data
data cached!


In [ ]:
# for this example, let's do multiprocessing with n_cores-2
import multiprocessing

n_cores = multiprocessing.cpu_count() - 2
n_cores = 24
print(f"using {n_cores} cores")


using 24 cores


In [ ]:

#this is our training env. It allows multiprocessing
#env_train, _ = e_train_gym.get_multiproc_env(n = n_cores)
env_train, _ = e_train_gym.get_sb_env()

#this is our observation environment. It allows full diagnostics
env_trade, _ = e_trade_gym.get_sb_env()

In [ ]:
agent = DRLAgent_sb3(env = env_train)

In [ ]:
TD3_PARAMS = {"batch_size": 64, 
              "buffer_size": 1000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 64, 'buffer_size': 1000, 'learning_rate': 0.001}
Using cuda device


In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=2000)

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1|   3|CASH SHORTAGE  |$34,701        |$99,210        |-8.53838% |-0.78977% |34.98%    
   1|   5|CASH SHORTAGE  |$22,879        |$99,846        |-7.43654% |-0.15409% |22.91%    
   1|   7|CASH SHORTAGE  |$15,790        |$99,774        |-6.32862% |-0.22558% |15.83%    
   1|  14|CASH SHORTAGE  |$3,908         |$99,261        |-4.02768% |-0.73947% |3.94%     
   1|  18|CASH SHORTAGE  |$15,273        |$99,621        |-2.49327% |-0.37910% |15.33%    
   1|  19|CASH SHORTAGE  |$18,037        |$99,214        |-2.22510% |-0.78565% |18.18%    
   1|  20|CASH SHORTAGE  |$40,193        |$98,736        |-1.01563% |-1.26432% |40.71%    
   1|  30|CASH SHORTAGE  |$45,722        |$97,670        |-0.50700% |-2.32996% |46.81%    
   1|  31|CASH SHORTAGE  |$46,348        |$98,069        |-0.46531% |-1.93121% |47.26%    
   1|  34|CASH SHORTAGE  |$18,333        |$99,357        |-1.23307% |

In [ ]:
df_account_value, df_actions = DRLAgent_sb3.DRL_prediction(model=trained_td3,
                        environment=e_train_gym)

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION


AssertionError: ignored

In [ ]:
# from torch.nn import Softsign, ReLU
ppo_params ={'n_steps': 256, 
             'ent_coef': 0.0, 
             'learning_rate': 0.0001, 
             'batch_size': 1024, 
            'gamma': 0.99}

policy_kwargs = {
    # "activation_fn": ReLU,
    "net_arch": [1024 for _ in range(10)], 
#     "squash_output": True
}

model = agent.get_model("ppo",  
                        model_kwargs = ppo_params, 
                        policy_kwargs = policy_kwargs, verbose = 0)

# model = model.load("scaling_reward.model", env = env_train)

{'n_steps': 256, 'ent_coef': 0.0, 'learning_rate': 0.0001, 'batch_size': 1024, 'gamma': 0.99}


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:138: UserWarning: You have specified a mini-batch size of 1024, but because the `RolloutBuffer` is of size `n_steps * n_envs = 256`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 256
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=256 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [ ]:
model.learn(total_timesteps = 100000) 
            # eval_env = env_trade, 
            # eval_freq = 500,
            # log_interval = 1, 
            # tb_log_name = 'env_cashpenalty_highlr',
            # n_eval_episodes = 1,
            # reset_num_timesteps = True)

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1|   1|TURBULENCE     |$77,346        |$99,735        |-0.26458% |-0.26458% |77.55%    
   1|   2|TURBULENCE     |$99,735        |$99,735        |-0.13229% |-0.26458% |100.00%   
   1|   3|TURBULENCE     |$99,735        |$99,735        |-0.08819% |-0.26458% |100.00%   
   1|   4|TURBULENCE     |$99,735        |$99,735        |-0.06614% |-0.26458% |100.00%   
   1|   5|TURBULENCE     |$99,735        |$99,735        |-0.05292% |-0.26458% |100.00%   
   1|   6|TURBULENCE     |$99,735        |$99,735        |-0.04410% |-0.26458% |100.00%   
   1|   7|TURBULENCE     |$99,735        |$99,735        |-0.03780% |-0.26458% |100.00%   
   1|   8|TURBULENCE     |$99,735        |$99,735        |-0.03307% |-0.26458% |100.00%   
   1|   9|TURBULENCE     |$99,735        |$99,735        |-0.02940% |-0.26458% |100.00%   
   1|  10|TURBULENCE     |$99,735        |$99,735        |-0.02646% |

ValueError: ignored

In [ ]:
trained_td3.save("different1_24.model")

In [ ]:
trade.head()

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=model,environment = e_trade_gym)